In [3]:
!pip install mediapipe opencv-python

In [4]:
import cv2
import mediapipe
import numpy
mp_drawing = mediapipe.solutions.drawing_utils  #drawing tools for drawing lines and such
mp_pose = mediapipe.solutions.pose  #pose estimation models

<h1>Camera Stuff</h1>

In [18]:
#Get video feed
camera = cv2.VideoCapture(0)  #number represents the image capture number
try:
    #50% confidence of detecting a body
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:  #mediapipe instance as pose
        while camera.isOpened():
            returned, frame = camera.read()

            #Detect poses
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #Reformat color to RGB
            image.flags.writeable = False  #make image read only
            
            results = pose.process(image)  #make detection stored as array

            image.flags.writeable = True  #re-allow modification to image
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #Reformat color to BGR

            #Render poses
            landmarks = results.pose_landmarks
            connections = mp_pose.POSE_CONNECTIONS
            lstyle = mp_drawing.DrawingSpec(color=(152,199,253), thickness=2, circle_radius=2)  #style for landmarks
            cstyle = mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)  #style for connections
                                        
            mp_drawing.draw_landmarks(image, landmarks, connections, lstyle, cstyle)  #draw landmarks
            
            
            cv2.imshow("Camera Feed", image)  #show camera feed
        
            if cv2.waitKey(10) & 0xFF == ord('q'):  #check if camera closed or quit
                break

finally:  #exit out
    camera.release()
    cv2.destroyAllWindows()

<h1>Joint Stuff</h1>

<img src='https://www.researchgate.net/publication/361071987/figure/fig2/AS:11431281214731862@1703733107226/33-Landmarks-detected-on-the-human-body-using-MediaPipe.png' alt='Labelled Joints Image' width='500'>

In [26]:
#Get video feed
camera = cv2.VideoCapture(0)  #number represents the image capture number
try:
    #50% confidence of detecting a body
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:  #mediapipe instance as pose
        while camera.isOpened():
            returned, frame = camera.read()

            #Detect poses
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #Reformat color to RGB
            image.flags.writeable = False  #make image read only
            
            results = pose.process(image)  #make detection stored as array

            image.flags.writeable = True  #re-allow modification to image
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #Reformat color to BGR

            #pinpoint landmarks
            try:
                landmarks = results.pose_landmarks.landmark

            except:
                pass

            #Render poses
            landmark_data = results.pose_landmarks
            connection_data = mp_pose.POSE_CONNECTIONS
            lstyle = mp_drawing.DrawingSpec(color=(152,199,253), thickness=2, circle_radius=2)  #style for landmarks
            cstyle = mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)  #style for connections
                                        
            mp_drawing.draw_landmarks(image, landmark_data, connection_data, lstyle, cstyle)  #draw landmarks
            
            
            cv2.imshow("Camera Feed", image)  #show camera feed
        
            if cv2.waitKey(10) & 0xFF == ord('q'):  #check if camera closed or quit
                break

finally:  #exit out
    camera.release()
    cv2.destroyAllWindows()

In [27]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]  #access value of right shoulder

x: 0.131724924
y: 0.767709136
z: -0.326806128
visibility: 0.999024808

In [28]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]  #access value of right elbow

x: 0.0353137031
y: 1.17583907
z: -0.157471612
visibility: 0.400015473

In [29]:
landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]  #access value of right wrist

x: 0.0589613318
y: 1.48322165
z: -0.493406713
visibility: 0.0873045921

<h1>Calculations For Angles</h1>

In [68]:
def calc_angle(first,second,third):
    first = numpy.array(first)  #first joint's coords
    second = numpy.array(second)  #second joint's coords
    third = numpy.array(third)  #third joint's coords

    #radians = [arctan of the vector b to c] - [arctan of the vector a to b]
    #if doing shoulder, elbow, wrist then it would be
    #radians = arctan(elbow[y]-wrist[y], elbow[x]-wrist[x]) - actan([shoulder[y]-elbow[y], shoulder[x]-elbow[x])
    radians = numpy.arctan2(third[1]-second[1], third[0]-second[0]) - numpy.arctan2(first[1]-second[1], first[0]-second[0])
    angle = numpy.abs(radians*180.0/numpy.pi)  #turn radians into angle

    if angle > 180.0:
        angle = 360-angle
    return angle

In [63]:
rShoulder_x = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x  #get x coords
rShoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y  #get y coords
rShoulder = [rShoulder_x, rShoulder_y]  #set rshoulder to be an array with x and y coords

In [64]:
rElbow_x = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x  #get x coords
rElbow_x = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y  #get y coords
rElbow = [rElbow_x, rElbow_x]  #set relbow to be an array with x and y coords

In [65]:
rWrist_x = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x  #get x coords
rWrist_x = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y  #get y coords
rWrist = [rWrist_x, rWrist_x]  #set rwrist to be an array with x and y coords

In [66]:
#calculate angle between shoulder elbow wrist
calc_angle(rShoulder,rElbow,rWrist)

10.446091445519249

In [70]:
# Get video feed
camera = cv2.VideoCapture(0)  # Number represents the image capture number
try:
    # 50% confidence of detecting a body
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:  # MediaPipe instance as pose
        while camera.isOpened():
            returned, frame = camera.read()

            # Detect poses
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Reformat color to RGB
            image.flags.writeable = False  # Make image read only

            results = pose.process(image)  # Make detection stored as array

            image.flags.writeable = True  # Re-allow modification to image
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Reformat color to BGR

            # Pinpoint landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Coords of the three landmarks trying to calculate
                rShoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                rElbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                rWrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                # Calculate angle between shoulder elbow wrist
                angle = calc_angle(rShoulder, rElbow, rWrist)

                # Print angle
                coords = tuple(numpy.multiply(rElbow, [700, 550]).astype(int))  # Determine positioning in relation to camera dimensions
                cv2.putText(image, str(angle), coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (202, 19, 12), 2, cv2.LINE_AA)

            except Exception as e:
                pass

            # Render poses
            landmark_data = results.pose_landmarks
            connection_data = mp_pose.POSE_CONNECTIONS
            lstyle = mp_drawing.DrawingSpec(color=(152, 199, 253), thickness=2, circle_radius=2)  # Style for landmarks
            cstyle = mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)  # Style for connections

            mp_drawing.draw_landmarks(image, landmark_data, connection_data, lstyle, cstyle)  # Draw landmarks

            cv2.imshow("Camera Feed", image)  # Show camera feed

            if cv2.waitKey(10) & 0xFF == ord('q'):  # Check if camera closed or quit
                break

finally:  # Exit out
    camera.release()
    cv2.destroyAllWindows()
